In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from pytorch_lightning import Trainer
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.core.lightning import LightningModule


from tqdm import tqdm

In [3]:
import sys
sys.path.append('../')

from dl4time.data.databuilder import StockDataset, StockData, StockData_ensemble
from dl4time.data.preprocessing import MinMaxTransformation, NormalizationCurrentDay

from dl4time.model.Conv import Conv1D
from dl4time.model.RNN import RNN
from dl4time.model.lighting import Model

from dl4time.utils.evaluation import Evaluator

In [4]:
class Model(LightningModule):
    def __init__(self, model):
        super().__init__()
        
        self.save_hyperparameters()
        
        self.model = model

    def forward(self, x):
        return self.model(x)
    
    def train_dataloader(self):
        params = {'batch_size': 1024,
                  'shuffle': True,
                  'num_workers': 4}
        
        train_dataset = StockDataset(data_all.data['x_train'], data_all.data['y_train'])
        
        return DataLoader(train_dataset, **params)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.00001)
    
#     def mse_loss(self):
#         loss = torch.nn.MSELoss(size_average=True)
#         return loss
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.mse_loss(logits, y)
        # add logging
        logs = {'loss': loss}
        return {'loss': loss, 'log': logs}
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.mse_loss(logits, y)
        return {'val_loss': loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

    def val_dataloader(self):
        params = {'batch_size': 1024,
                  'shuffle': False,
                  'num_workers': 4}
        
        val_dataset = StockDataset(data_all.data['x_val'], data_all.data['y_val'])
        return DataLoader(val_dataset, **params)
        
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.mse_loss(logits, y)
        return {'val_loss': loss}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss}
        return {'test_loss': avg_loss, 'log': tensorboard_logs}

    def test_dataloader(self):
        params = {'batch_size': 1024,
                  'shuffle': False,
                  'num_workers': 4}
        
        test_dataset = StockDataset(data_all.data['x_test'], data_all.data['y_test'])
        return DataLoader(test_dataset, **params)

In [11]:
dates = pd.date_range('2000-01-02','2017-10-11',freq='B')

import pickle
with open('../Data_Folder/Stocks_all.pickle', 'rb') as f:
    data_all = pickle.load(f)

In [14]:
net2 = Conv1D(in_channels=1, output_dim=1)
model_Conv1D = Model(net2)

trainer = Trainer(max_epochs=100, gpus=1)
trainer.fit(model_Conv1D)

trainer.test()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | Conv1D | 242 K 


Epoch 1:  87%|████████▋ | 8783/10038 [04:16<00:36, 34.19it/s, loss=10.904, v_num=32]  
Validating: 0it [00:00, ?it/s]
Epoch 2:  87%|████████▋ | 8783/10038 [04:15<00:36, 34.32it/s, loss=11.016, v_num=32]  
Validating: 0it [00:00, ?it/s]
Epoch 2:  92%|█████████▏| 9249/10038 [04:19<00:22, 35.69it/s, loss=11.016, v_num=32]


Epoch 3:  87%|████████▋ | 8783/10038 [04:10<00:35, 34.99it/s, loss=14.197, v_num=32]  
Validating: 0it [00:00, ?it/s]
Epoch 3:  98%|█████████▊| 9828/10038 [04:17<00:05, 38.18it/s, loss=14.197, v_num=32]


Epoch 4:  87%|████████▋ | 8783/10038 [04:09<00:35, 35.17it/s, loss=12.144, v_num=32]  
Validating: 0it [00:00, ?it/s]
Epoch 5:  87%|████████▋ | 8783/10038 [04:09<00:35, 35.15it/s, loss=10.897, v_num=32]  
Validating: 0it [00:00, ?it/s]
Epoch 5:  91%|█████████ | 9114/10038 [04:12<00:25, 36.13it/s, loss=10.897, v_num=32]


Epoch 6:  87%|████████▋ | 8783/10038 [04:09<00:35, 35.17it/s, loss=11.880, v_num=32]  
Validating: 0it [00:00, ?it/s]
Epoch 6:  96%|█████████▋| 9681/10038 [04:15<00:09, 37.92it/s, loss=11.880, v_num=32]


...

Epoch 97:  87%|████████▋ | 8783/10038 [03:59<00:34, 36.67it/s, loss=13.118, v_num=32] 
Validating: 0it [00:00, ?it/s]
Epoch 97:  98%|█████████▊| 9867/10038 [04:05<00:04, 40.16it/s, loss=13.118, v_num=32]


Epoch 98:  87%|████████▋ | 8783/10038 [03:59<00:34, 36.69it/s, loss=12.045, v_num=32] 
Validating: 0it [00:00, ?it/s]
Epoch 99:  87%|████████▋ | 8783/10038 [03:59<00:34, 36.66it/s, loss=11.132, v_num=32]  
Validating: 0it [00:00, ?it/s]
Epoch 99:  92%|█████████▏| 9200/10038 [04:02<00:22, 37.98it/s, loss=11.132, v_num=32]


Epoch 100:  87%|████████▋ | 8783/10038 [03:59<00:34, 36.70it/s, loss=18.862, v_num=32] 
Validating: 0it [00:00, ?it/s]
Epoch 100:  98%|█████████▊| 9821/10038 [04:05<00:05, 40.06it/s, loss=18.862, v_num=32]


Testing: 100%|█████████▉| 2506/2510 [00:13<00:00, 188.92it/s]--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(39.8863, device='cuda:0')}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 2510/2510 [00:14<00:00, 178.91it/s]


{'test_loss': 39.88628387451172}

In [20]:
symbol = "aapl"
stock = StockData(symbol, dates, transformation=NormalizationCurrentDay())
stock.inference(model_Conv1D)

stock_evaluator = Evaluator()
stock_evaluator.evaluate(pd.Series(stock.y_pred['test'].flatten()), pd.Series(stock.y_true['test'].flatten()))
print(stock_evaluator.metrics)

stock.plot()

/home/shun/Desktop/projects/DLforTime/venv/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

../dl4time/utils/evaluation.py:28: RuntimeWarning:

invalid value encountered in double_scalars



explained variance           0.001473
MAE                          1.009701
MSE                          2.056060
MedAE                        0.702735
RSQ                          0.000100
accuracy                    54.280510
edge                              inf
noise                     1331.980347
y_true_chg                        inf
y_pred_chg                 679.217773
prediction_calibration       0.000000
capture_ratio                     NaN
edge_long                         NaN
edge_short                        NaN
edge_win                          NaN
edge_lose                         NaN
dtype: float64


In [16]:
symbol = "ibm"
stock = StockData(symbol, dates, transformation=NormalizationCurrentDay())
stock.inference(model_Conv1D)

stock_evaluator = Evaluator()
stock_evaluator.evaluate(pd.Series(stock.y_pred['test'].flatten()), pd.Series(stock.y_true['test'].flatten()))
print(stock_evaluator.metrics)

stock.plot()

/home/shun/Desktop/projects/DLforTime/venv/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



explained variance         -0.001701
MAE                         0.812279
MSE                         1.346964
MedAE                       0.593524
RSQ                        -0.004286
accuracy                   50.175439
edge                             NaN
noise                     160.230698
y_true_chg                       inf
y_pred_chg                 92.682594
prediction_calibration      0.000000
capture_ratio                    NaN
edge_long                        NaN
edge_short                       NaN
edge_win                         NaN
edge_lose                        NaN
dtype: float64


In [24]:
symbol = "AMD"
stock = StockData(symbol, dates, transformation=NormalizationCurrentDay())
stock.inference(model_Conv1D)

stock_evaluator = Evaluator()
stock_evaluator.evaluate(pd.Series(stock.y_pred['test'].flatten()), pd.Series(stock.y_true['test'].flatten()))
print(stock_evaluator.metrics)

stock.plot()

/home/shun/Desktop/projects/DLforTime/venv/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



explained variance         -0.002224
MAE                         2.568671
MSE                        16.339312
MedAE                       1.738052
RSQ                        -0.003515
accuracy                   52.564103
edge                             NaN
noise                     284.238434
y_true_chg                       inf
y_pred_chg                163.032089
prediction_calibration      0.000000
capture_ratio                    NaN
edge_long                        NaN
edge_short                       NaN
edge_win                         NaN
edge_lose                        NaN
dtype: float64
